<a href="https://colab.research.google.com/github/moh2-lh/moh2-lh/blob/main/Copy_of_02_Cross_Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

© 2021 Zaka AI, Inc. All Rights Reserved

#Cross Validation

**Objective**: The goal of this notebook is to learn how to evaluate the performance of a trained neural network using cross validation. Cross validation splits the dataset into K-folds or partitions and repeats the train test procedure on them where one fold is chosen for testing and the rest for training.

We will be cross validating our model that had been trained on the pima dataset of the previous notebook using both manual and automatic approaches.

In [7]:
# clone git repo
!git clone https://github.com/zaka-ai/intro2dl.git

# change directory
%cd intro2dl/data/

Cloning into 'intro2dl'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 16 (delta 1), reused 7 (delta 0), pack-reused 0
Unpacking objects: 100% (16/16), done.
/content/intro2dl/data/intro2dl/data/intro2dl/data


## Manual Cross Validation

### 1. Load data & split variables
The dataset is the same as the previous notebook. It has eight features and one output as result of diabetes onset.

In [8]:
# Pima Indians Dataset with 10-fold cross validation
import numpy

# load pima indians dataset
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")

# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]

### 2. Define train function

Define a `train_model` function that takes `x_train`, `y_train`, `x_test` and `y_test`  as input and does the following:

1.    Create a model
2.    Compile the model
3.    Fit the model on `x_train` and `y_train`
4.    Evaluate the model on `x_test` and `y_test`
5.    Print & return accuracy

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def train_model(x_train, y_train, x_test, y_test):
  # FILL BLANKS
  # create model
  model = Sequential()
  model.add(Dense(12, input_dim=8, activation='relu'))
  model.add(Dense(10, activation='relu'))
  model.add(Dense(1 , activation='sigmoid'))

	# Compile model
  model.compile(optimizer='sgd', loss= 'binary_crossentropy', metrics=['accuracy']) 

  # Fit the model

  model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs= 10 , batch_size= 10, verbose=0)

	
  # evaluate the model

  scores = model.evaluate(x_test, y_test, verbose=0)


  print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
  
  return scores[1] * 100

### 3. Put everything together

Write the code that will split the dataset into 10 different folds, loop over them and train the models.

In [23]:
from sklearn.model_selection import StratifiedKFold

# define 10-fold cross validation test harness
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=12)
cvscores = []

for train, test in kfold.split(X, Y):
	accuracy = train_model(X[train], Y[train], X[test], Y[test])
	cvscores.append(accuracy)


print("%.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores)))



accuracy: 64.94%
accuracy: 64.94%
accuracy: 62.34%
accuracy: 74.03%
accuracy: 64.94%
accuracy: 64.94%
accuracy: 64.94%
accuracy: 61.04%
accuracy: 65.79%
accuracy: 65.79%
65.37% (+/- 3.23%)


## Automatic Cross Validation
### Scikit-learn & Keras

The Keras library provides a convenient wrapper for deep learning models to be used as classification or regression estimators in scikit-learn.



*   KerasClassifier
*   KerasRegressor



In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
import numpy

# Function to create & compile model, required for KerasClassifier
def create_model():
    model = Sequential()
    model.add(Dense(12, input_dim=8, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
  
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  
    return model

# load pima indians dataset
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")

# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]

# create model
model = KerasClassifier(build_fn=create_model, epochs=150, batch_size=10, verbose=0)

# evaluate using 10-fold cross validation
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=12)
results = cross_val_score(model, X, Y, cv=kfold)
print("%.2f%%" % (results.mean() * 100))

71.23%
